In [ ]:
#------------------------------------------------------------------------
# Example: Sequencing
#
# Demonstrates the use of SignalFlow nodes to generate continuous 
# sequences. For more complex sequences that can be modified on-the-fly,
# use isobar: https://github.com/ideoforms/isobar
#------------------------------------------------------------------------

from signalflow import *
import numpy as np

graph = AudioGraph()

In [ ]:
class SquareArpPatch (Patch):
    #------------------------------------------------------------------------
    # Simple patch that iterates through a sequence of MIDI notes
    # following the given clock
    #------------------------------------------------------------------------
    def __init__(self, clock, sequence_array, fundamental, pan=0.0):
        super().__init__()
        sequence = Sequence(np.array(sequence_array), clock)
        frequency = MidiNoteToFrequency(sequence + fundamental)
        square = SquareOscillator(frequency, width=SineLFO([0.1, 0.11], 0.05, 0.5))
        filter_env = ScaleLinExp(ASREnvelope(0.001, 0.01, 0.1, clock=clock) ** 3, 0, 1, 100, 5000)
        filtered = SVFilter(square, "low_pass", filter_env, resonance=0.2) * 0.2
        balanced = StereoBalance(filtered, pan)
        self.set_output(balanced)

class HiHatPatch (Patch):
    #------------------------------------------------------------------------
    # Minimum viable hi-hat
    #------------------------------------------------------------------------
    def __init__(self, clock, amp=1.0):
        super().__init__()
        noise = WhiteNoise()
        noise = SVFilter(noise, "high_pass", 8000)
        env = ASREnvelope(0.001, 0.01, 0.1, clock=clock)
        pan = SineLFO(0.5, -0.3, 0.3)
        output = StereoPanner(noise * env * amp, pan)
        self.set_output(output)

#------------------------------------------------------------------------
# Create a 16th-beat clock
#------------------------------------------------------------------------
bpm = 110
clock = Impulse(4 * bpm/60)

#------------------------------------------------------------------------
# Shared sequence that is used to transpose MIDI notes
#------------------------------------------------------------------------
fundamental = Sequence([0, -4, -5], Impulse(1/4))

#------------------------------------------------------------------------
# Create three parallel arpeggios
#------------------------------------------------------------------------
bass = SquareArpPatch(clock, np.array([ 0, 0, 0, 0, 7, 12, 0, 7 ]) + 36, fundamental)
bass.play()

seq1 = SquareArpPatch(clock, np.array([ 3, 2, 0, -2, 0]) + 60, fundamental, pan=-0.75)
seq1.play()

seq2 = SquareArpPatch(clock, np.array([ 7, 3, 5, 2, 3, 0 ]) + 72, fundamental, pan=0.75)
seq2.play()

#------------------------------------------------------------------------
# Add a simple hi-hat sequence
#------------------------------------------------------------------------
amp_sequence = Sequence([ 0.1, 0.25, 0.5, 0.1 ], clock=clock)
patch = HiHatPatch(clock, amp=amp_sequence)
patch.play()

In [ ]:
graph.clear()